In [9]:
#构造一个可接受任意数量参数的函数

#接受任意数量参数的位置参数
def avg(first,*rest):
    #rest是一个tuple
    return (first+sum(rest))/(1+len(rest))

print(avg(1,2))
print(avg(1,2,3,4))

#接受任意数量参数的关键字参数
def printDict(name,value,**attrs):
    #attrs是一个字典
    keyvals=['%s=%s'%item for item in attrs.items()]
    attrvals=' '.join(keyvals)
    print('{}:{:.2f},has attr:{}'.format(name,value,attrvals))

printDict('p',2.3456,dtype='float',size=32)


#总结
#注意点:`**`参数只能出现在最后一个参数，而`*`参数后面仍然可以定义其他函数,但`*`必须是最后一个位置参数
#*args,这里args相当于一个tuple,**kwargs这里kwargs相当于一个dict
def a(x,*args,y):
    pass
def b(x,*args,y,**kwargs):
    print(x)
    print(args)
    print(y)
    print(kwargs)

b(1,2,3,4,y=3,key=2,size=3,name='w')

def anyattr(*args,**kwargs):
    print(args) #a tuple
    print(kwargs) #a dict

1.5
2.5
p:2.35,has attr:dtype=float size=32
1
(2, 3, 4)
3
{'key': 2, 'size': 3, 'name': 'w'}


In [11]:
#强制关键字参数

#将强制关键字参数放在某个`*`参数或者单个`*`后面就能达到这种效果

def recv(maxsize,*,block):
    print(maxsize)
    print(block)

# recv(1024,True) #TypeError
recv(1024,block=True)

1024
True


In [15]:
# 使用函数参数注解

#python解释器不会对这里的类型进行检测，运行时和没有加注解时没有任何差距，对于阅读源码的人很有帮助
def add(x:int,y:int)->int:
    return x+y

help(add)

Help on function add in module __main__:

add(x: int, y: int) -> int



In [17]:
#定义带默认值的函数
def spam(a,b=42):
    print(a,b)
spam(1)
spam(1,2)

#测试某个默认参数是否有传递进来
_no_value=object()
def spam(a,b=_no_value):
    if b is _no_value:
        print('No b value supplied')

spam(1)
spam(1,2)
spam(1,None)

#特别注意：默认参数的值应该是不可变的对象，比如None,True,False,数字或者字符串，千万不要使用可变对象.


1 42
1 2
No b value supplied


In [20]:
#lambda表达式在定义时绑定值

#为绑定值，自由变量x在运行时确定
x=10
a=lambda y:x+y
x=20
b=lambda y:x+y
print(a(10))
print(b(10))

#绑定自由变量，使其在定义时固定
x=10
a=lambda y,x=x:x+y
x=20
b=lambda y,x=x:x+y
print(a(10))
print(b(10))

30
30
20
30


In [29]:
#将只__init__的类转化为一个函数--->闭包
#一个闭包就是一个函数，只不过在函数内部带上了一个额外的变量环境。

#例子
from urllib.request import urlopen

# class UrlTemplate:
#     def __init__(self,template):
#         self.template=template
#     def open(self,**kwargs):
#         return urlopen(self.template.format_map(kwargs))

# yahoo=UrlTemplate('http://finance.yahoo.com/d/quotes.csv?s={names}&f={fields}')
# for line in yahoo.open(names='IBM,AAPL,FB', fields='sl1c1v'):
#     print(line.decode('utf-8'))

#将上述类转为闭包
def urltemplate(template):
    def opener(**kwargs):
        return urlopen(template.format_map(kwargs))
    return opener

baidu=urltemplate('https://www.baidu.com/s?ie={ie}&wd={wd}')
for line in baidu(ie='UTF-8',wd='123'):
    print(line.decode('utf-8'))

<html>

<head>

	<script>

		location.replace(location.href.replace("https://","http://"));

	</script>

</head>

<body>

	<noscript><meta http-equiv="refresh" content="0;url=http://www.baidu.com/"></noscript>

</body>

</html>


In [35]:
#回调函数的使用

#定义一个需要回调函数的函数
def apply_async(func,args,*,callback):
    result=func(*args)
    
    callback(result)

def print_result(result):
    print('Got:',result)
def add(x,y):
    return x+y

apply_async(add,(1,2),callback=print_result)
apply_async(add,('xx','yy'),callback=print_result)

#定义一个可以访问外部信息的回调函数
#方法一：定义一个类
class ResultHandler:
    def __init__(self):
        self._sequence=0
    def handler(self,result):
        self._sequence+=1
        print('[{}] Got:{}'.format(self._sequence,result))

r=ResultHandler()
apply_async(add,(1,2),callback=r.handler)
apply_async(add,('xx','yy'),callback=r.handler)

#方法二：定义一个闭包
def make_handler():
    sequence=0
    def handler(result):
        #要修改内部变量的值需要使用nonlocal声明
        nonlocal sequence
        sequence+=1
        print('[{}] Got:{}'.format(sequence,result))
    return handler

handler=make_handler()
apply_async(add,(1,2),callback=handler)
apply_async(add,('xxx','yyy'),callback=handler)

Got: 3
Got: xxyy
[1] Got:3
[2] Got:xxyy
[1] Got:3
[2] Got:xxxyyy


In [39]:
# 访问闭包中定义的变量

def sample():
    n=0
    def func():
        print('n=',n)
    def get_n():
        return n
    def set_n(value):
        #要修改内部变量的值需要使用
        nonlocal n
        n=value
    #绑定
    func.get_n=get_n
    func.set_n=set_n
    return func

f=sample()
print(f())
f.set_n(10)
print(f())
print(f.get_n())

n= 0
None
n= 10
None
10
